# Text Classification


In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import RandomizedSearchCV
from sklearn import metrics
import re , itertools
from IPython.display import display
from sklearn.feature_extraction.text import TfidfTransformer
import numpy as np
from os import walk
from io import StringIO
import pandas as pd

In [2]:
#Read
dir_name = 'Dataset' #Folder Dataset
df_list = []
df = pd.DataFrame(columns=['Time','Title', 'Content', 'label'])
for (dirpath, dirnames, filenames) in walk(dir_name):
    for label in dirnames: 
        for (dirpath, dirnames, filenames) in walk(dir_name+'/'+label):
            for filename in filenames:
                f = open(dir_name+'/'+label+'/'+filename, "r")
                f = StringIO(f.read().replace('\n', '\t') + '\t' + label) #replace new line with tab
                temp = pd.read_csv(f, sep="\t", names=['Time','Title', 'Content', 'label'])
                df_list.append(temp)
                
            break
    break
df = pd.concat(df_list)
df = df.reset_index()
df

,index,Time,Title,Content,label
0,0,"Kamis, 25 September 2008 | 19:46 WIB",RI dan Myanmar Bicarakan Bank Langsung,"JAKARTA, KAMIS - Perwakilan RI di Myanma...",ekonomi
1,0,"Senin, 22 September 2008 | 04:09 WIB",Pemerintah Alirkan Rp 60 Triliun ke Perbankan,"JAKARTA, SENIN - Pemerintah akan mengalirkan d...",ekonomi
2,0,"Rabu, 15 Oktober 2008 | 08:42 WIB",Dana Buy Back Baru Terpakai Rp 8 Miliar,"JAKARTA, RABU - Badan usaha milik negara hanya...",ekonomi
3,0,"Senin, 13 Oktober 2008 | 13:17 WIB","Masih Prihatin, Saham Taiwan Ditutup Turun","TAIPEI, SENIN - Harga saham Taiwan ditutup tur...",ekonomi
4,0,"Kamis, 16 Oktober 2008 | 06:45 WIB",Saham Selandia Baru Turun,"WELLINGTON, KAMIS - Saham-saham Selandia Baru ...",ekonomi
...,...,...,...,...,...
460,0,"Selasa, 23 September 2008 | 09:40 WIB",Lesehan Tegal Rasa Bintang Lima,"Menyebut nama Tegal, mungkin yang langsung And...",travel
461,0,"Kamis, 18 September 2008 | 17:00 WIB","Pempek Abing: Garing, Gurih, dan Kenyal",Ingin makan tapi tidak mau makanan yang berat-...,travel
462,0,"Selasa, 14 Oktober 2008 | 09:07 WIB",Masakan Tajin Maroko,"Orang Maroko suka tajin? Ya, iyalah! Tetapi, y...",travel
463,0,"Rabu, 24 September 2008 | 17:15 WIB",Layanan Plus untuk Wisatawan Lebaran di Yogyak...,"YOGYAKARTA, RABU - Badan Pariwisata Daerah (Ba...",travel


In [3]:
News = []
for x in df['Content']:
  #Convert to lower case
  x = x.lower()
  #Convert www.* or https?://* to URL
  x = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',x)
  #Convert @username to AT_USER
  x = re.sub('@+','',x)
  #Remove additional white spaces
  x = re.sub('[\s]+', ' ', x)
  #Replace #word with word
  x = re.sub(r'#([^\s]+)', r'\1', x)
  #trim
  x = x.strip('\'"')
  News.append(str(x))


In [4]:
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

stopword = StopWordRemoverFactory().create_stop_word_remover()
stemmer = StemmerFactory().create_stemmer()

corpus = []

for i,kalimat in enumerate(News):
  stop = stopword.remove(kalimat)
  stem = stemmer.stem(stop)
  print('loading kalimat ke:',i+1,'dari',len(News))
  corpus.append(stem)
df['Content'] = corpus

loading kalimat ke: 1 dari 465
loading kalimat ke: 2 dari 465
loading kalimat ke: 3 dari 465
loading kalimat ke: 4 dari 465
loading kalimat ke: 5 dari 465
loading kalimat ke: 6 dari 465
loading kalimat ke: 7 dari 465
loading kalimat ke: 8 dari 465
loading kalimat ke: 9 dari 465
loading kalimat ke: 10 dari 465
loading kalimat ke: 11 dari 465
loading kalimat ke: 12 dari 465
loading kalimat ke: 13 dari 465
loading kalimat ke: 14 dari 465
loading kalimat ke: 15 dari 465
loading kalimat ke: 16 dari 465
loading kalimat ke: 17 dari 465
loading kalimat ke: 18 dari 465
loading kalimat ke: 19 dari 465
loading kalimat ke: 20 dari 465
loading kalimat ke: 21 dari 465
loading kalimat ke: 22 dari 465
loading kalimat ke: 23 dari 465
loading kalimat ke: 24 dari 465
loading kalimat ke: 25 dari 465
loading kalimat ke: 26 dari 465
loading kalimat ke: 27 dari 465
loading kalimat ke: 28 dari 465
loading kalimat ke: 29 dari 465
loading kalimat ke: 30 dari 465
loading kalimat ke: 31 dari 465
loading kalimat k

loading kalimat ke: 253 dari 465
loading kalimat ke: 254 dari 465
loading kalimat ke: 255 dari 465
loading kalimat ke: 256 dari 465
loading kalimat ke: 257 dari 465
loading kalimat ke: 258 dari 465
loading kalimat ke: 259 dari 465
loading kalimat ke: 260 dari 465
loading kalimat ke: 261 dari 465
loading kalimat ke: 262 dari 465
loading kalimat ke: 263 dari 465
loading kalimat ke: 264 dari 465
loading kalimat ke: 265 dari 465
loading kalimat ke: 266 dari 465
loading kalimat ke: 267 dari 465
loading kalimat ke: 268 dari 465
loading kalimat ke: 269 dari 465
loading kalimat ke: 270 dari 465
loading kalimat ke: 271 dari 465
loading kalimat ke: 272 dari 465
loading kalimat ke: 273 dari 465
loading kalimat ke: 274 dari 465
loading kalimat ke: 275 dari 465
loading kalimat ke: 276 dari 465
loading kalimat ke: 277 dari 465
loading kalimat ke: 278 dari 465
loading kalimat ke: 279 dari 465
loading kalimat ke: 280 dari 465
loading kalimat ke: 281 dari 465
loading kalimat ke: 282 dari 465
loading ka

In [5]:
df[['Content']]

,Content
0,jakarta kamis - wakil ri myanmar laku temu man...
1,jakarta senin - perintah alir dana hingga rp 6...
2,jakarta rabu - badan usaha milik negara sempat...
3,taipei senin - harga saham taiwan tutup turun ...
4,wellington kamis - saham selandia baru turun 3...
...,...
460,sebut nama tegal mungkin langsung ingat usaha ...
461,makan mau makan berat coba pempek empek khazan...
462,orang maroko suka tajin ya iya tetapi maksud t...
463,yogyakarta rabu - badan pariwisata daerah bapa...


In [6]:
sentences = df['Content'].values
y = df['label'].values
# sentences_train, sentences_test, y_train, y_test = train_test_split(sentences, y, test_size=0.25, random_state=42)

In [7]:
cv = CountVectorizer()
cou_vec = cv.fit_transform(sentences)
tfidf = TfidfTransformer()
corps = tfidf.fit_transform(cou_vec)

indices = np.arange(corps.shape[0])
X_train, X_test, y_train, y_test, idx1, idx2 = train_test_split(corps, y, indices, test_size=0.33, random_state=42)

In [8]:
classifier = MultinomialNB()
classifier.fit(X_train, y_train)
Y_pred = classifier.predict(X_test)


# result = pd.DataFrame(columns=['Time','Title', 'Content', 'Pred','True'])



print('accuracy: ', metrics.accuracy_score(y_test, Y_pred))
print(metrics.confusion_matrix(y_test,Y_pred))

accuracy:  0.974025974025974
[[39  2  0  0  0]
 [ 0 28  0  0  0]
 [ 0  0 18  0  0]
 [ 0  0  0 28  0]
 [ 2  0  0  0 37]]


In [9]:
df_pred = pd.DataFrame({'Pred': Y_pred[:]})
result = pd.concat([df.iloc[idx2].reset_index(), df_pred], axis=1)

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(result[['Time', 'Title', 'label', 'Pred']])

,Time,Title,label,Pred
0,"Rabu, 15 Oktober 2008 | 10:51 WIB",Bank Dunia Dituding Penyebar Multikrisis Global,ekonomi,ekonomi
1,"Senin, 13 Oktober 2008 | 11:47 WIB",Penjaminan Simpanan Naik Jadi 20 Kali Lipat,ekonomi,ekonomi
2,"Senin, 13 Oktober 2008 | 14:55 WIB",Semua Rupiah Harus Ditarik ke Indonesia,ekonomi,ekonomi
3,"Kamis, 25 September 2008 | 15:29 WIB",Pengunjung Borobudur Ditargetkan Capai 200.000,travel,travel
4,"Rabu, 15 Oktober 2008 | 14:18 WIB",Memasang Instalasi Lampu Taman,properti,properti
5,"Sabtu, 11 Oktober 2008 | 23:23 WIB",Kemenangan di Depan Mata Sebastien Loeb,olahraga,olahraga
6,"Senin, 13 Oktober 2008 | 10:11 WIB",Kursi-kursi Lucu dan Unik,properti,properti
7,"Kamis, 16 Oktober 2008 | 07:25 WIB",Perusahaan Sekuritas Harus Tingkatkan Risk Man...,ekonomi,ekonomi
8,"Kamis, 25 September 2008 | 06:25 WIB",Titik Nol (39): Kremasi,travel,travel
9,"Kamis, 25 September 2008 | 23:23 WIB",Taufik Bertemu Bao Chunlai di Perempat Final,olahraga,olahraga
